In [0]:


df_bronze = spark.read.parquet("/Volumes/comunicacao/comunicacao/bronze/stream_dados.parquet/")
display(df_bronze)



In [0]:
df_celular = df_bronze.filter(df_bronze['Dispositivo de acesso'] == 'Celular')
display(df_celular)

In [0]:
from pyspark.sql.functions import col

def padronizar_colunas(df_celular):
    novas_colunas = [c.replace(" ", "_") for c in df_celular.columns]
    for antiga, nova in zip(df_celular.columns, novas_colunas):
        df_celular = df_celular.withColumnRenamed(antiga, nova)
    return df_celular


In [0]:
df_cel = padronizar_colunas(df_celular)


In [0]:
display(df_cel)

In [0]:
# Nome da tabela (registrada no metastore)
nome_tabela = "comunicacao.bronze.bronze_celular"

# Caminho físico onde os dados Delta são armazenados
caminho_delta = "/Volumes/comunicacao/comunicacao/bronze/bronze_celular"

# Dados novos (df_novos pode vir de transformação, leitura, etc.)
# Exemplo:
# df_novos = spark.read.parquet("...")  ou gerado dinamicamente
df_novos = spark.createDataFrame([], schema="...")  # substitua com seu DataFrame real

# Verifica se a tabela existe no metastore
if spark._jsparkSession.catalog().tableExists(nome_tabela):
    print("✅ Tabela já existe. Inserindo novos dados (append).")
    df_novos.write \
        .format("delta") \
        .mode("append") \
        .save(caminho_delta)
else:
    print("📌 Tabela não existe. Criando com overwrite.")
    df_novos.write \
        .format("delta") \
        .mode("overwrite") \
        .save(caminho_delta)

    # Registra no catálogo (opcional, mas recomendado)
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {nome_tabela}
        USING DELTA
        LOCATION '{caminho_delta}'
    """)


In [0]:

df_cel.write.format("delta").mode("overwrite").save("/Volumes/comunicacao/comunicacao/bronze/bronze_celular")

In [0]:
df = spark.read.format("delta").load("/Volumes/comunicacao/comunicacao/bronze/bronze_celular/")
display(df)


